In [30]:


from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


    
html = "https://www.yelp.com/search?find_desc=fine+dining&find_loc=Sacramento,+CA&start=0" 
page = requests.get(html)
soup = BeautifulSoup(page.content, 'html.parser')

#get all the names of the restaurants
names = soup.find('meta', property = 'og:description')['content']
names = names.split("-")[1]
names = names.split(",")


#get the descriptions
desc = []
for i in range(10, 23):
    p = soup.find_all("p")[i]
    desc.append(p)
 

desc = [element.get_text().strip() for element in desc]
desc = [re.sub("read more", "", element) for element in desc]
desc = [element.replace("\n", "") for element in desc]

del desc[0]
del desc[10]
del desc[-1]




#get all phone numbers
phone_numbers =  soup.find_all('span', {'class' : 'biz-phone'})
each_number = [number.get_text().strip() for number in phone_numbers]

#get all reviews
reviews = soup.find_all('span', {'class': 'review-count rating-qualifier'} )
all_reviews = [review.get_text().strip() for review in reviews]
all_reviews = [re.sub("reviews", "", element) for element in all_reviews]


#get all addresses
addresses  = soup.find_all('address')
addresses  = [address.get_text().strip() for address in addresses]

#get all of part of SacTown
final_addresses =  soup.find_all('span', {'class': 'neighborhood-str-list'})
final_add = [add.get_text().strip() for add in final_addresses]
final_add.insert(7, "Sacramento")
final_add.insert(1, "Sacramento")

#putting it all together
all_info = pd.DataFrame(dict(names = names, descriptions = desc, reviews = all_reviews, addresses = 
                            addresses, town = final_add, numbers = each_number))




all_info.head(4)













,names,descriptions,reviews,addresses,town,numbers
0,Mulvaney's B & L,“used to fine dining and had a tough time deci...,1003,1215 19th St,Midtown,(916) 441-6022
1,The Kitchen Restaurant,“Its fine dining as best as it can get in the ...,678,2225 Hurley Way,Sacramento,(916) 568-7171
2,Ella Dining Room and Bar,"“are seasonal, which demonstrated fresh produc...",1557,1131 K St,Downtown,(916) 443-3772
3,The Firehouse Restaurant,“We were looking for a fine dining experience ...,1190,1112 2nd St,Old Sacramento,(916) 442-4772
